In [2]:
import pandas as pd 
import numpy as np
import os

from pandas.api.types import CategoricalDtype
from plotnine import *
from plotnine.data import mpg
%matplotlib inline

import matplotlib.pyplot as plt

In [3]:
path = os.getcwd()
loop_up_table_path = path + '/COVID-19/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv'
covid_us_confirmed_path = path + '/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

In [4]:
# load data
land = pd.read_csv("land.csv")
census = pd.read_csv("final_census_imputed.csv")
loop_up_table = pd.read_csv(loop_up_table_path)
covid_us_confirmed = pd.read_csv(covid_us_confirmed_path)
state_name = pd.read_csv('state_name.csv')

In [5]:
# get the fips code for census
fips_list = []
for i in range(len(census)):
    fips = str(float(census.iloc[i]['GEO_ID'][-5:]))
    fips_list.append(fips)

In [6]:
census['FIPS_merge_on'] = fips_list

In [7]:
# Get FIPS code for covid data
covid_fips_list = []
for i in range(len(covid_us_confirmed)):
    covid_fips = covid_us_confirmed.iloc[i]['FIPS']
    covid_fips_list.append(str(covid_fips))

In [8]:
covid_us_confirmed["FIPS_merge_on"] = covid_fips_list

In [9]:
covid_us_confirmed

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,FIPS_merge_on
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,...,0,0,0,0,0,0,0,0,0,60.0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,...,151,151,151,152,152,152,154,154,154,66.0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,...,16,16,19,19,19,19,19,21,21,69.0
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.220800,-66.590100,...,2173,2198,2256,2299,2329,2427,2542,2589,2646,72.0
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.335800,-64.896300,...,68,69,69,69,69,69,69,69,69,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3256,84070016,US,USA,840,NaN,Central Utah,Utah,US,39.372319,-111.575868,...,27,28,28,28,29,29,29,29,30,nan
3257,84070017,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,...,13,13,14,13,13,13,13,14,14,nan
3258,84070018,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,...,155,164,170,167,180,187,200,209,219,nan
3259,84070019,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,...,16,16,16,16,16,16,16,19,19,nan


In [10]:
# get full names of each state for land
full_name_list = []
for i in range(len(land)):
    code = land.iloc[i]["States"].strip()
    full_name = state_name[state_name.Code == code]["State"].iloc[0]
    full_name_list.append(full_name)  

In [11]:
land['state_full_name'] = full_name_list

In [12]:
# get combined_key to merge on for land 
combined_key_list = []
for i in range(len(land)):
    combined_key = land.iloc[i]["County"] + ", " + land.iloc[i]["state_full_name"] + ', US'
    combined_key_list.append(combined_key)

In [13]:
land["Combined_Key"] = combined_key_list

In [14]:
# drop the unwanted rows for land 
land_final = land[["LAND AREA(SQMI)","Combined_Key"]]

In [15]:
# merge land and covid
middle_data = pd.merge(land_final,covid_us_confirmed, on='Combined_Key', how='right')

In [16]:
middle_data.shape

(3266, 130)

In [17]:
# merge census and middle_data
final_data = pd.merge(census, middle_data, on='FIPS_merge_on', how='outer')

In [21]:
final_data.to_csv('final_data_update.csv',index = False)

In [18]:
final_data.shape

(3266, 230)

In [19]:
final_data

,Unnamed: 0,GEO_ID,COUNTY,STATE,POPULATION,AGE15_19,AGE20_24,AGE25_29,AGE30_34,AGE55_59,...,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20
0,0.0,0500000US01001,Autauga County,Alabama,55200.0,7.0,6.0,6.6,6.2,7.5,...,67,68,74,84,91,93,104,103,110,110
1,1.0,0500000US01003,Baldwin County,Alabama,208107.0,6.1,5.2,5.7,5.5,6.9,...,208,216,222,224,227,231,243,244,254,254
2,2.0,0500000US01005,Barbour County,Alabama,25782.0,5.6,6.4,7.2,7.0,6.4,...,53,58,59,61,67,69,74,79,79,81
3,3.0,0500000US01007,Bibb County,Alabama,22527.0,6.7,6.6,6.9,6.7,6.6,...,44,45,46,46,46,46,46,49,50,50
4,4.0,0500000US01009,Blount County,Alabama,57645.0,6.4,5.5,5.9,5.9,6.8,...,44,44,44,45,45,45,45,45,45,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69,56,40,38,57,73,110,125,112,101
3262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [20]:
land[land.States == " AK"]

,Unnamed: 0,County,States,LAND AREA(SQMI),state_full_name,Combined_Key
67,67,Aleutians East,AK,8496.37,Alaska,"Aleutians East, Alaska, US"
68,68,Aleutians West,AK,8718.39,Alaska,"Aleutians West, Alaska, US"
69,69,Anchorage,AK,1792.36,Alaska,"Anchorage, Alaska, US"
70,70,Bethel,AK,41642.96,Alaska,"Bethel, Alaska, US"
71,71,Bristol Bay,AK,644.05,Alaska,"Bristol Bay, Alaska, US"
72,72,Denali,AK,6379.26,Alaska,"Denali, Alaska, US"
73,73,Dillingham,AK,23882.34,Alaska,"Dillingham, Alaska, US"
74,74,Fairbanks North Star,AK,7393.10,Alaska,"Fairbanks North Star, Alaska, US"
75,75,Haines,AK,2478.88,Alaska,"Haines, Alaska, US"
76,76,Hoonah-Angoon,AK,11736.33,Alaska,"Hoonah-Angoon, Alaska, US"


In [372]:
dnull = final_data[final_data['LAND AREA(SQMI)'].isnull() == True] 
dnull = dnull[dnull['GEO_ID'].isnull() == False]

In [373]:
dnull

,Unnamed: 0,GEO_ID,COUNTY,STATE,POPULATION,AGE15_19,AGE20_24,AGE25_29,AGE30_34,AGE55_59,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
69,69.0,0500000US02020,Anchorage Municipality,Alaska,296112.0,6.2,8.2,9.2,8.3,6.6,...,67,73,81,85,88,98,103,109,112,121
74,74.0,0500000US02090,Fairbanks North Star Borough,Alaska,99653.0,6.2,10.9,10.2,8.2,6.0,...,40,42,46,53,53,65,71,73,76,76
78,78.0,0500000US02122,Kenai Peninsula Borough,Alaska,58220.0,6.3,5.7,6.2,6.2,8.2,...,8,10,11,12,13,13,13,14,15,15
81,81.0,0500000US02158,Kusilvak Census Area,Alaska,8198.0,9.1,8.0,8.8,6.6,5.5,...,0,0,0,0,0,0,0,0,0,0
83,83.0,0500000US02170,Matanuska-Susitna Borough,Alaska,103464.0,6.9,6.3,6.9,7.3,7.1,...,4,4,4,4,4,4,5,6,9,10
319,319.0,0500000US11001,District of Columbia,District of Columbia,684498.0,5.5,7.9,12.0,11.3,5.3,...,653,757,902,1002,1097,1211,1440,1523,1660,1778
332,332.0,0500000US12027,DeSoto County,Florida,36399.0,6.8,5.9,7.1,7.2,5.5,...,8,9,11,12,14,16,18,19,20,20
480,480.0,0500000US13189,McDuffie County,Georgia,21498.0,6.5,6.1,6.2,6.1,6.7,...,4,5,5,6,8,13,16,18,19,20
644,644.0,0500000US17099,LaSalle County,Illinois,110401.0,6.2,6.0,6.0,6.1,7.7,...,6,6,7,8,11,11,17,17,19,18
704,704.0,0500000US18015,Carroll County,Indiana,19994.0,6.6,5.2,5.3,5.0,7.5,...,2,2,3,3,3,3,3,3,4,5


In [314]:
land.shape

(3145, 6)

In [315]:
covid_us_confirmed.shape

(3253, 93)

In [316]:
middle_data.shape

(3258, 94)

In [317]:
census.shape

(3142, 101)

In [353]:
final_data.shape

(3258, 194)